In [21]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('/Users/weizheng/Downloads/all/train.csv')
df_train.head()

ID      target  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  \
0  000d6aaf2  38000000.0        0.0          0        0.0          0   
1  000fbd867    600000.0        0.0          0        0.0          0   
2  0027d6b71  10000000.0        0.0          0        0.0          0   
3  0028cbf45   2000000.0        0.0          0        0.0          0   
4  002a68644  14400000.0        0.0          0        0.0          0   

   2f0771a37  30347e683  d08d1fbe3  6ee66e115    ...      3ecc09859  \
0          0          0          0          0    ...            0.0   
1          0          0          0          0    ...            0.0   
2          0          0          0          0    ...            0.0   
3          0          0          0          0    ...            0.0   
4          0          0          0          0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0   
1        0.0        0.0          0          0          0          0   
2        0.0        0.0          0          0          0          0   
3        0.0        0.0          0          0          0          0   
4        0.0        0.0          0          0          0          0   

   fb36b89d9  7e293fbaf  9fc776466  
0          0          0          0  
1          0          0          0  
2          0          0          0  
3          0          0          0  
4          0          0          0  

[5 rows x 4993 columns]

In [78]:
df_train['target'].describe()

count    4.459000e+03
mean     5.944923e+06
std      8.234312e+06
min      3.000000e+04
25%      6.000000e+05
50%      2.260000e+06
75%      8.000000e+06
max      4.000000e+07
Name: target, dtype: float64

In [67]:
df_train.drop('target', axis=1).loc[df_train.ID=='000d6aaf2']

ID  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  2f0771a37  \
0  000d6aaf2        0.0          0        0.0          0          0   

   30347e683  d08d1fbe3  6ee66e115  20aa07010    ...      3ecc09859  \
0          0          0          0        0.0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0   

   fb36b89d9  7e293fbaf  9fc776466  
0          0          0          0  

[1 rows x 4992 columns]

In [12]:
df_test = pd.read_csv('/Users/weizheng/Downloads/all/test.csv')
print(df_test.shape)

(49342, 4992)


In [15]:
df_test.head()

ID  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  2f0771a37  \
0  000137c73        0.0        0.0        0.0        0.0        0.0   
1  00021489f        0.0        0.0        0.0        0.0        0.0   
2  0004d7953        0.0        0.0        0.0        0.0        0.0   
3  00056a333        0.0        0.0        0.0        0.0        0.0   
4  00056d8eb        0.0        0.0        0.0        0.0        0.0   

   30347e683  d08d1fbe3  6ee66e115  20aa07010    ...      3ecc09859  \
0        0.0        0.0        0.0        0.0    ...            0.0   
1        0.0        0.0        0.0        0.0    ...            0.0   
2        0.0        0.0        0.0        0.0    ...            0.0   
3        0.0        0.0        0.0        0.0    ...            0.0   
4        0.0        0.0        0.0        0.0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0        0.0        0.0        0.0        0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0   
2        0.0        0.0        0.0        0.0        0.0        0.0   
3        0.0        0.0        0.0        0.0        0.0        0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0   

   fb36b89d9  7e293fbaf  9fc776466  
0        0.0        0.0        0.0  
1        0.0        0.0        0.0  
2        0.0        0.0        0.0  
3        0.0        0.0        0.0  
4        0.0        0.0        0.0  

[5 rows x 4992 columns]

In [3]:
df_train.shape

(4459, 4993)

In [17]:
X_data = df_train.drop(['ID', 'target'], axis=1)
y_data = df_train['target']
test_X = df_test.drop('ID', axis=1)

In [56]:
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [79]:
X_train,X_test,y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [80]:
X_train = np.asarray(X_train)
y_train = np.log1p(np.asanyarray(y_train))
X_test = np.asarray(X_test)
y_test = np.log1p(np.asanyarray(y_test))
test_X = np.asarray(test_X)

In [81]:
y_train[:5]

array([ 15.20180517,  11.67844838,  13.77051424,  12.89922233,  15.42494867])

In [46]:
print(X_train.shape)
print(y_train.shape)

(3121, 4991)
(3121,)


In [49]:
print(X_test.shape)
print(y_test.shape)

(1338, 4991)
(1338,)


In [4]:
import lightgbm as lgb

In [75]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=200, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [82]:
%time pred_y, model_lgb, evals_result = run_lgb(X_train, y_train, X_test, y_test, test_X)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 1.45788
[400]	valid_0's rmse: 1.43298
[600]	valid_0's rmse: 1.42972
Early stopping, best iteration is:
[625]	valid_0's rmse: 1.42909
CPU times: user 2min 40s, sys: 6.81 s, total: 2min 47s
Wall time: 58.6 s


In [83]:
pred_y

array([ 14.38842374,  14.28961455,  14.00241425, ...,  13.53321074,
        13.45857826,  14.41556027])

In [61]:
import pickle

In [84]:
#save it
with open('lgb.pickle','wb') as f:
    pickle.dump(model_lgb,f)

## random forest

In [57]:
rf = RandomForestRegressor()

In [58]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [59]:
rf.feature_importances_

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         1.27653562e-04,   4.09741461e-05,   3.26858963e-07])

In [70]:
def get_target_and_features(cust_id):
    sel = df_train.loc[df_train.ID==cust_id].drop('ID', axis=1)
    target = sel['target'].values
    features = sel.drop('target', axis=1).values
    return target, features

In [71]:
tar, features = get_target_and_features('002a68644')

In [72]:
rf.predict(features)

array([ 8868000.])

In [87]:
from sklearn.model_selection import GridSearchCV

In [ ]:
estimator = GridSearchCV(pipe,
                         dict(pca__n_components=n_components,
                         clf__C=Cs), n_jobs=-1)
estimator.fit(X_train, y_train.codes)